In [ ]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher
from spacy.tokens import Span
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.set_option('display.max_colwidth', 200)
%matplotlib inline

ModuleNotFoundError: ignored

In [ ]:
print(res)

[c. mackenzie, craig vincent, craig di francia, sebastian maniscalco, paul ben-victor, robbie robertson, kota eberhardt, selene gallio, simon kinberg, jessica chastain, mauro fiore, kyle chandler, millie bobby brown, bradley whitford, sally hawkins, charles dance, thomas middleditch, aisha hinds, o'shea jackson jr., david strathairn, ken watanabe, zhang ziyi, vivienne graham, madison disowns, joe morton, ghidorah, scott chambliss, paul blart, sophie, douglas hodge, samuel beckett, beckett, alan schneider, jack macgowran, buster keaton, david rayner clark, max wall, beckett, beckett, chris columbus, alfonso cuarón, mike newell, david yates, david heyman, barron, tanya seghatchian, harry potter, harry potter, berg, david ayer, bruce wayne, miller, barry allen, pedro pascal, geoff johns, chris brancato, michael green, marc guggenheim, dan mazeau, john francis daley, jonathan goldstein, peter safran, berg, david ayer, bruce wayne, miller, barry allen, pedro pascal, geoff johns, chris branc

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving wiki_sentences_v2.csv to wiki_sentences_v2.csv


In [ ]:
sentence_file = pd.read_csv("wiki_sentences_v2.csv")

In [ ]:
sentence_file['sentence'].sample(5)


832                               12 exposure rolls have been used widely in the daily press.
3967    the characterization of the autobot twins, skids and mudflap, was heavily criticized.
3615                                    slavi refuses, dismissing mccall as old and impotent.
3897               the film's costume designers are rama rajamouli and prasanthi tripuraneni.
275                           others include duck soup  and national lampoon's animal house .
Name: sentence, dtype: object

In [ ]:
process_sent = nlp("12 exposure rolls have been used widely in the daily press")

for sent in process_sent:
  print(sent.text+"......"+sent.dep_)

12......nummod
exposure......compound
rolls......nsubjpass
have......aux
been......auxpass
used......ROOT
widely......advmod
in......prep
the......det
daily......amod
press......pobj


In [ ]:
def getEnt(pass_Sent):
  ent1=""
  ent2=""
  prev_sent_text=""
  prev_sent_dep=""

  prefix=""
  modifier=""

  for token in nlp(pass_Sent):
    if token.dep_ != "punct":
      if token.dep_=="compound":
        prefix = token.text
        if prev_sent_text == "compound":
          prefix = prev_sent_text + " " + token.text
      if token.dep_.endswith("mod")==True:
        modifier = token.text
        if prev_sent_dep=="compound":
          modifier = prev_sent_text + " " + token.text
      if token.dep_.find("subj")==True:
        ent1 = modifier + " " + prefix + " " + token.text
        prefix=""
        modifier=""
        prev_sent_text=""
        prev_sent_dep=""

      if token.dep_.find("obj")==True:
         ent2 = modifier + " " + prefix + " " + token.text
      prev_sent_text = token.text
      prev_sent_dep = token.dep_

  return [ent1.strip(), ent2.strip()]

In [ ]:
'''res = []
nlp = spacy.load('en')
for i in sentence_file["sentence"]:                                                                                                               
  sents = nlp(i) 
  [res.append(ee) for ee in sents.ents if ee.label_ == 'PERSON']'''

In [ ]:
nlp = spacy.load()
sentm = nlp("Suman is Fucking hot girl.")

for i in sentm:
  print(i.dep_)

nsubj
aux
ROOT
amod
dobj
punct


In [ ]:
ent_pairs = []

for pass_Sent in sentence_file["sentence"]:
  ent_pairs.append(getEnt(pass_Sent))
print(ent_pairs)

[['connie', 'own'], ['later  woman', 'distance'], ['christian', 'then  elder'], ['temple', 'fire'], ['outside  cult', 'him'], ['it', 'religious  awakening'], ['c. mackenzie', 'craig cast'], ['later di francia', 'action cast'], ['sebastian  maniscalco', 'later ben cast'], ['we', 'just  film'], ['we', 'tests'], ['m global', 'international sales rights'], ['robbie robertson', 'soundtrack'], ['it', 'original music tracks'], ['it', 'reviewed  franchise'], ['she', 'accidentally  mystique'], ['', 'military  arrest'], ['train', 'vuk'], ['telepath', 'gallio'], ['singer', 'men'], ['last  which', 'penn'], ['jessica chastain', 'same  character'], ['mauro fiore', 'cinematographer'], ['filming', 'october'], ['soundtrack', 'digitally  june'], ['album', 'digitally  august'], ['film', 'motion pictures'], ['it', '$  103.7'], ['', ''], ['it', 'executive yoshimitsu banno'], ['vivienne graham', 'former employee dr'], ['emma monster which', 'several monarch dr'], ['madison', 'mythological  texts'], ['joe mo

In [ ]:
def getVerb(Pass_sent):
  doc = nlp(Pass_sent)

  match = Matcher(nlp.vocab)
  pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'ADJ','OP':"?"}]

  match.add("matching_1", None, pattern)
  matching = match(doc)

  k = len(matching)-1
  spn = doc[matching[k][1]:matching[k][2]]
  return spn.text




In [ ]:
list_root = [getVerb(i) for i in sentence_file['sentence'] ]

In [ ]:
pd.Series(list_root).value_counts()[:50]

is               370
was              297
released on       87
include           73
are               71
were              71
released          40
's                38
composed by       35
has               31
have              31
became            31
become            29
released in       27
included          26
called            22
produced          22
considered        20
made              20
had               20
been              18
be                18
used              17
scheduled         17
stars             16
directed by       16
written by        15
wrote             14
hired             14
set               14
produced by       14
received          13
went              13
introduced in     13
wanted            12
won               11
began in          11
began             11
gave              10
features          10
sold              10
reported           9
includes           9
opened             9
used in            9
shot in            9
produced in        9
directed     

In [ ]:
subject = [i[0] for i in ent_pairs]
objec = [i[1] for i in ent_pairs]
fin_df = pd.DataFrame({'source':subject, 'target':objec, "relations":list_root})
fin_df

,source,target,relations
0,connie,own,decides
1,later woman,distance,heard in
2,christian,then elder,paralyzed by
3,temple,fire,set on
4,outside cult,him,wails with
...,...,...,...
4313,confidencial,negatively film,responded
4314,le parisien,five star rating,gave
4315,museum collection,"37,000 film titles",includes
4316,predecessor,historical film 1946,was


In [ ]:
G=nx.from_pandas_edgelist(fin_df, "source", "target",
edge_attr=True, create_using=nx.DiGraph())

In [ ]:
plt.figure(figsize=(12,12))
pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

KeyboardInterrupt: ignored

<Figure size 864x864 with 0 Axes>

In [ ]:
btw_cent = nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None)

In [ ]:
Pg_Rank_cent = nx.pagerank(G, alpha=0.85, personalization=None, tol=1e-06, nstart=None, weight=None, dangling=None)
print(Pg_Rank_cent)

{'connie': 0.00014292826565714275, 'own': 0.0002647210630426053, 'later  woman': 0.00014292826565714275, 'distance': 0.0002647210630426053, 'christian': 0.0002647210630426053, 'then  elder': 0.0003673650730207423, 'temple': 0.00014292826565714275, 'fire': 0.0002647210630426053, 'outside  cult': 0.00014292826565714275, 'him': 0.0019609182260790594, 'it': 0.0018576505530994126, 'religious  awakening': 0.0001500998106582364, 'c. mackenzie': 0.00014292826565714275, 'craig cast': 0.0002647210630426053, 'later di francia': 0.00014292826565714275, 'action cast': 0.0002647210630426053, 'sebastian  maniscalco': 0.00014292826565714275, 'later ben cast': 0.0002647210630426053, 'we': 0.00014292826565714275, 'just  film': 0.0001484643019019365, 'tests': 0.0001484643019019365, 'm global': 0.00014292826565714275, 'international sales rights': 0.0002647210630426053, 'robbie robertson': 0.00014292826565714275, 'soundtrack': 0.0005407135102305505, 'original music tracks': 0.0001500998106582364, 'reviewe

In [ ]:
nlp = spacy.load('en')
for i in btw_cent.keys():
  nlp = spacy.load('en')                                                                                                                  
  sents = nlp(document_string) 
  #[del btw_cent[ee] for ee in sents.ents if ee.label_ != 'PERSON']
  for ee in sents.ents:
    if ee.label_ != 'PERSON':
      del btw_cent[ee]
  dict1 = OrderedDict(sorted(btw_cent.items()))
  print("Most Famous Personality: "+ dicct1[len(dict1)-1])
     

NameError: ignored